In [1]:
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from itertools import combinations
import seaborn as sns
from sklearn import metrics
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.feature_selection import RFE
from bayes_opt import BayesianOptimization

In [6]:
data=pd.read_excel(r"C:\Users\HP\Desktop\Data.xlsx",sheet_name='16+3',index_col=0,header=0)
data1=data.iloc[0:18, :]    
OP=data1[['lg(O3)','lg(H2O2)','pH','TOC','Φn4','O3(l)20']]  
minmax_scaler=preprocessing.MinMaxScaler()  
data2=minmax_scaler.fit_transform(OP)
data3=pd.DataFrame(data2,columns=['lg(O3)','lg(H2O2)','pH','TOC','Φn4','O3(l)20'])  

In [7]:
X_full=data3[['lg(O3)','lg(H2O2)','pH','Φn4','O3(l)20']]  
y_full=data3['TOC']

def run_bayesian_optimization(X_train_all, y_train_all):
    def black_box_function(learning_rate, n_estimators, min_samples_split, max_features, max_depth, max_leaf_nodes):
        model = GradientBoostingRegressor(
            learning_rate=learning_rate,
            n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999),
            max_depth=int(max_depth),
            max_leaf_nodes=int(max_leaf_nodes),
            random_state=2
        )
        loo = LeaveOneOut()
        y_real, y_predicted = [], []
        for train_index, test_index in loo.split(X_train_all):
            X_train, X_val = X_train_all[train_index], X_train_all[test_index]
            y_train, y_val = y_train_all[train_index], y_train_all[test_index]
            model.fit(X_train, y_train.ravel())
            y_pred = model.predict(X_val)
            y_real.append(y_val[0])
            y_predicted.append(y_pred[0])
        return r2_score(y_real, y_predicted)
    
    pbounds = {
        'learning_rate': (0.001, 0.2),
        'n_estimators': (10, 500),
        'min_samples_split': (2, 25),
        'max_features': (1, 4),
        'max_depth': (1, 5),
        'max_leaf_nodes': (2, 15)
    }
    optimizer = BayesianOptimization(
        f=black_box_function,
        pbounds=pbounds,
        random_state=1
    )
    optimizer.maximize(init_points=15, n_iter=20)
    return optimizer.max['params']


test_scores = []
test_rmse_scores = []
n_runs = 10  

for run in range(n_runs):
    
    X_train_all, X_test, y_train_all, y_test = train_test_split(
    X_full.values,  
    y_full.values, 
    test_size=4, 
    random_state=run
)
    
    best_params = run_bayesian_optimization(X_train_all, y_train_all)
    
    final_model = GradientBoostingRegressor(
        learning_rate=best_params['learning_rate'],
        n_estimators=int(best_params['n_estimators']),
        max_leaf_nodes=int(best_params['max_leaf_nodes']),
        max_features=min(best_params['max_features'], 0.999),
        min_samples_split=int(best_params['min_samples_split']),
        max_depth=int(best_params['max_depth']),
        random_state=2
    )
    final_model.fit(X_train_all, y_train_all.ravel())
    y_pred = final_model.predict(X_test)

    test_r2 = r2_score(y_test, y_pred)
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred)) 
    
    test_scores.append(test_r2)
    test_rmse_scores.append(test_rmse)  
    
    print(f"Run {run+1}/10 - Test R²: {test_r2:.4f}, Test RMSE: {test_rmse:.4f}")  


mean_r2 = np.mean(test_scores)
std_r2 = np.std(test_scores)
mean_rmse = np.mean(test_rmse_scores)  
std_rmse = np.std(test_rmse_scores)    

print(f"\n Average test R²score: {mean_r2:.4f} ± {std_r2:.4f}")
print(f"verage testRMSE: {mean_rmse:.4f} ± {std_rmse:.4f}")

|   iter    |  target   | learni... | max_depth | max_fe... | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.6991    | 0.08399   | 3.881     | 1.0       | 5.93      | 5.375     | 55.25     |
| 2         | 0.5868    | 0.03807   | 2.382     | 2.19      | 9.005     | 11.64     | 345.8     |
| 3         | 0.5798    | 0.04169   | 4.512     | 1.082     | 10.72     | 11.6      | 283.8     |
| 4         | 0.6361    | 0.02894   | 1.792     | 3.402     | 14.59     | 9.209     | 349.2     |
| 5         | 0.486     | 0.1754    | 4.578     | 1.255     | 2.508     | 5.906     | 440.3     |
| 6         | -0.1598   | 0.02057   | 2.684     | 3.874     | 8.931     | 17.91     | 164.6     |
| 7         | -0.1598   | 0.1376    | 4.339     | 1.055     | 11.75     | 24.74     | 376.6     |
| 8         | -0.1598   | 0.05681   | 4.157     | 1.31      | 7.823     | 22.9      | 153.9     |
| 9         | 0.6534

| 10        | -0.1598   | 0.09882   | 1.213     | 2.722     | 3.907     | 15.55     | 352.9     |
| 11        | 0.6102    | 0.02136   | 2.656     | 3.083     | 7.384     | 3.149     | 272.6     |
| 12        | -0.1598   | 0.1331    | 3.06      | 3.834     | 9.625     | 22.78     | 77.36     |
| 13        | -0.1598   | 0.02872   | 4.23      | 2.193     | 4.15      | 23.33     | 180.4     |
| 14        | -0.1598   | 0.1504    | 3.904     | 3.65      | 10.11     | 19.27     | 181.0     |
| 15        | -0.1598   | 0.05472   | 4.584     | 2.284     | 14.54     | 17.26     | 314.6     |
| 16        | 0.2383    | 0.001     | 4.03      | 1.0       | 14.37     | 3.897     | 339.9     |
| 17        | 0.6306    | 0.03615   | 5.0       | 1.0       | 11.08     | 13.23     | 272.8     |
| 18        | 0.6244    | 0.2       | 1.0       | 4.0       | 2.0       | 13.15     | 277.5     |
| 19        | 0.593     | 0.06211   | 1.0       | 1.0       | 13.19     | 2.0       | 128.2     |
| 20        | -0.007

| 20        | -0.07196  | 0.001     | 5.0       | 4.0       | 13.25     | 2.0       | 54.73     |
| 21        | 0.6553    | 0.1283    | 1.0       | 1.0       | 3.775     | 2.372     | 51.29     |
| 22        | 0.6272    | 0.2       | 1.0       | 1.0       | 10.38     | 2.0       | 42.91     |
| 23        | 0.6671    | 0.1287    | 4.883     | 3.006     | 11.97     | 3.029     | 279.8     |
| 24        | 0.617     | 0.02804   | 1.722     | 3.918     | 14.34     | 9.723     | 274.6     |
| 25        | 0.5004    | 0.2       | 1.0       | 4.0       | 3.39      | 6.94      | 280.4     |
| 26        | 0.575     | 0.2       | 1.097     | 1.0       | 2.0       | 5.149     | 62.35     |
| 27        | -0.1598   | 0.2       | 5.0       | 1.0       | 5.798     | 14.03     | 271.3     |
| 28        | 0.1029    | 0.001     | 1.0       | 4.0       | 15.0      | 2.0       | 272.3     |
| 29        | 0.1095    | 0.001     | 1.0       | 4.0       | 15.0      | 8.042     | 282.6     |
| 30        | 0.2048

| 30        | 0.2995    | 0.001     | 5.0       | 4.0       | 12.79     | 2.0       | 447.2     |
| 31        | 0.4345    | 0.2       | 5.0       | 1.0       | 15.0      | 2.0       | 279.5     |
| 32        | 0.4345    | 0.2       | 5.0       | 4.0       | 15.0      | 2.0       | 135.3     |
| 33        | -0.00403  | 0.001     | 1.0       | 1.0       | 2.0       | 2.0       | 141.9     |
| 34        | 0.5913    | 0.2       | 1.0       | 4.0       | 2.0       | 2.0       | 47.53     |
| 35        | -0.1598   | 0.001     | 1.0       | 4.0       | 2.0       | 15.2      | 277.1     |
Run 4/10 - Test R²: 0.8233, Test RMSE: 0.0844
|   iter    |  target   | learni... | max_depth | max_fe... | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.3369    | 0.08399   | 3.881     | 1.0       | 5.93      | 5.375     | 55.25     |
| 2         | 0.5375    | 0.03807   | 2.382     | 2.19      | 9.005     

| 3         | 0.5412    | 0.04169   | 4.512     | 1.082     | 10.72     | 11.6      | 283.8     |
| 4         | 0.54      | 0.02894   | 1.792     | 3.402     | 14.59     | 9.209     | 349.2     |
| 5         | 0.4978    | 0.1754    | 4.578     | 1.255     | 2.508     | 5.906     | 440.3     |
| 6         | -0.1598   | 0.02057   | 2.684     | 3.874     | 8.931     | 17.91     | 164.6     |
| 7         | -0.1598   | 0.1376    | 4.339     | 1.055     | 11.75     | 24.74     | 376.6     |
| 8         | -0.1598   | 0.05681   | 4.157     | 1.31      | 7.823     | 22.9      | 153.9     |
| 9         | 0.5635    | 0.05827   | 1.52      | 1.058     | 10.82     | 6.867     | 140.1     |
| 10        | -0.1598   | 0.09882   | 1.213     | 2.722     | 3.907     | 15.55     | 352.9     |
| 11        | 0.6749    | 0.02136   | 2.656     | 3.083     | 7.384     | 3.149     | 272.6     |
| 12        | -0.1598   | 0.1331    | 3.06      | 3.834     | 9.625     | 22.78     | 77.36     |
| 13        | -0.159

| 13        | -0.1598   | 0.02872   | 4.23      | 2.193     | 4.15      | 23.33     | 180.4     |
| 14        | -0.1598   | 0.1504    | 3.904     | 3.65      | 10.11     | 19.27     | 181.0     |
| 15        | -0.1598   | 0.05472   | 4.584     | 2.284     | 14.54     | 17.26     | 314.6     |
| 16        | 0.5458    | 0.04668   | 5.0       | 1.0       | 10.24     | 13.44     | 269.3     |
| 17        | 0.2071    | 0.001     | 4.925     | 1.0       | 13.44     | 2.0       | 339.1     |
| 18        | 0.5556    | 0.2       | 1.0       | 4.0       | 2.0       | 10.61     | 276.7     |
| 19        | 0.5555    | 0.2       | 1.0       | 4.0       | 15.0      | 2.0       | 279.2     |
| 20        | 0.5547    | 0.2       | 1.0       | 4.0       | 2.263     | 2.0       | 259.7     |
| 21        | 0.6405    | 0.0506    | 4.433     | 1.0       | 3.463     | 2.0       | 40.45     |
| 22        | -0.1006   | 0.001     | 1.0       | 1.0       | 15.0      | 11.56     | 42.22     |
| 23        | 0.5544

| 23        | 0.7041    | 0.06223   | 3.886     | 1.0       | 5.051     | 2.0       | 47.08     |
| 24        | 0.7122    | 0.104     | 4.882     | 1.0       | 13.61     | 2.0       | 49.07     |
| 25        | 0.5071    | 0.1473    | 3.081     | 1.0       | 10.21     | 10.31     | 45.47     |
| 26        | -0.1126   | 0.001     | 1.0       | 4.0       | 12.44     | 2.0       | 39.88     |
| 27        | 0.5943    | 0.2       | 5.0       | 1.0       | 15.0      | 2.0       | 57.08     |
| 28        | -0.09376  | 0.001     | 5.0       | 1.0       | 15.0      | 10.58     | 53.56     |
| 29        | 0.4433    | 0.2       | 1.0       | 3.683     | 9.141     | 2.0       | 52.04     |
| 30        | 0.4574    | 0.2       | 5.0       | 1.0       | 2.0       | 8.182     | 48.17     |
| 31        | 0.6467    | 0.2       | 5.0       | 1.0       | 7.306     | 2.0       | 62.54     |
| 32        | 0.5943    | 0.2       | 5.0       | 1.0       | 14.66     | 2.0       | 67.51     |
| 33        | -0.073

| 33        | 0.2596    | 0.001     | 4.641     | 4.0       | 11.55     | 3.628     | 348.3     |
| 34        | 0.2018    | 0.001     | 2.557     | 1.0       | 11.19     | 6.233     | 352.0     |
| 35        | 0.5834    | 0.2       | 4.43      | 4.0       | 9.334     | 2.0       | 272.4     |
Run 9/10 - Test R²: 0.8763, Test RMSE: 0.0873
|   iter    |  target   | learni... | max_depth | max_fe... | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.6526    | 0.08399   | 3.881     | 1.0       | 5.93      | 5.375     | 55.25     |
| 2         | 0.6547    | 0.03807   | 2.382     | 2.19      | 9.005     | 11.64     | 345.8     |
| 3         | 0.6412    | 0.04169   | 4.512     | 1.082     | 10.72     | 11.6      | 283.8     |
| 4         | 0.6436    | 0.02894   | 1.792     | 3.402     | 14.59     | 9.209     | 349.2     |
| 5         | 0.6271    | 0.1754    | 4.578     | 1.255     | 2.508     